In [1]:
import seisbench.data as sbd
import seisbench.generate as sbg
from seisLM.model import pretrained_models
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
import torch
import einops
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


In [2]:
data = sbd.STEAD()
waveforms = data.get_waveforms(1265656)
input_values = torch.Tensor(waveforms).unsqueeze(0)

2024-06-17 18:53:36,068 | seisbench | WARNING | Component order not specified, defaulting to 'ZNE'.


In [3]:
# sinkhorn
# model_name_or_path = '/scicore/home/dokman0000/liu0003/projects/seisLM/results/models/pretrained_seisLM/42__2024-06-17-07h-24m-11s_model_config_4xdownsample_sinkhorn/checkpoints/epoch=4-step=87350.ckpt'

# no sinkhorn
model_name_or_path = '/scicore/home/dokman0000/liu0003/projects/seisLM/results/models/pretrained_seisLM/42__2024-06-17-07h-24m-09s_model_config_4xdownsample/checkpoints/epoch=5-step=104820.ckpt'
model = pretrained_models.LitMultiDimWav2Vec2.load_from_checkpoint(
    model_name_or_path
)

model.train()
model.model(input_values = input_values.cuda())

codevector_probs.shape [B * L, G, V]: torch.Size([1499, 2, 320])
codevector_probs sum over B*L [[  0   0   0   0   0   0   0   0   0   0   0   0   0 409   0 105   0   0
    0   0   0   0   0   0   0   0   2   1   0   0   0   0   0   0   0   0
    0   0   0   1  38   0   0   0   0   0   0   0   0   1  16   0   0   0
    0   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0 131
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   0
    6   0   1   0   0   0   0 198   0   0   0   0   0   0   0   0   0  14
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  19   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   3   0   0   0   0   0   0   0   0   0   0   4   0   0   0
    0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    1   1   0   0

Wav2Vec2ForPreTrainingOutput(loss=None, projected_states=tensor([[[-5.3812e-02, -2.5643e-01, -3.4578e-01,  ...,  2.4930e-01,
           7.4815e-01,  1.2085e+00],
         [ 3.3131e-01,  2.7996e-01, -2.0832e-01,  ..., -7.0221e-04,
           5.0616e-01,  1.3651e+00],
         [ 3.3153e-01,  4.8283e-01, -2.6049e-02,  ..., -3.0532e-01,
           1.8925e-01,  1.3003e+00],
         ...,
         [ 3.2181e-02,  5.6427e-01, -1.2699e-02,  ...,  1.4149e-01,
          -5.8645e-02,  7.4689e-01],
         [ 1.9687e-02,  1.2502e-01, -5.1004e-01,  ...,  3.0885e-01,
           6.4124e-02,  6.1751e-01],
         [ 2.1214e-01, -3.2704e-01, -8.0461e-01,  ...,  1.2750e-01,
          -9.4340e-01,  8.4187e-01]]], device='cuda:0',
       grad_fn=<ViewBackward0>), projected_quantized_states=tensor([[[ 0.0326,  0.2522,  0.1149,  ..., -0.2531,  0.4117,  0.4596],
         [-0.1083,  0.0632, -0.0061,  ...,  0.3773, -0.1067,  0.2311],
         [ 0.0954, -0.0181, -0.1022,  ...,  0.4110, -0.1951,  0.0502],
       

In [4]:
# # tensor([[[117.4146,  91.8246,  89.4750,  ..., -36.3809, -38.6474, -40.9750],
# #          [110.1232,  97.5682,  91.9317,  ..., -40.5390, -40.9350, -48.4647]],

# #         [[165.3371,  66.3771,  63.0904,  ..., -43.2461, -46.6159, -50.4394],
# #          [149.8903,  99.6392,  81.1701,  ..., -40.9141, -41.9816, -45.8662]],

# #         [[196.7281,  63.7894,  60.3995,  ..., -46.7848, -49.4018, -51.2654],
# #          [181.5987,  64.0762,  53.6267,  ..., -42.6589, -43.2662, -48.0746]],

# #         ...,

# #         [[ 98.2172,  89.8766,  50.7380,  ..., -39.3649, -43.1647, -51.2411],
# #          [126.5686, 120.8874, 102.4355,  ..., -40.8563, -42.1127, -47.2328]],

# #         [[122.0727, 116.9310,  87.2223,  ..., -52.6367, -64.8259, -73.5588],
# #          [147.6193, 138.6167, 102.2152,  ..., -46.0260, -49.5940, -60.3351]],

# #         [[160.0314,  89.6809,  67.1128,  ..., -41.1726, -41.2953, -49.0211],
# #          [136.1050, 108.9605,  79.3012,  ..., -50.8108, -52.8083, -57.9031]]],
# #        device='cuda:0', grad_fn=<SortBackward0>)
# # tensor([[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
# #           0.0000e+00, 0.0000e+00],
# #          [8.0855e-43, 0.0000e+00, 4.2039e-45,  ..., 0.0000e+00,
# #           0.0000e+00, 1.8794e-37]],

# #         [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
# #           0.0000e+00, 0.0000e+00],
# #          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
# #           0.0000e+00, 0.0000e+00]],

# #         [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
# #           0.0000e+00, 0.0000e+00],
# #          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
# #           0.0000e+00, 0.0000e+00]],

# #         ...,

# #         [[1.1694e-41, 8.4078e-45, 0.0000e+00,  ..., 0.0000e+00,
# #           1.0706e-42, 0.0000e+00],
# #          [0.0000e+00, 2.8026e-45, 0.0000e+00,  ..., 0.0000e+00,
# #           0.0000e+00, 0.0000e+00]],

# #         [[8.7701e-37, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
# #           0.0000e+00, 0.0000e+00],
# #          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
# #           0.0000e+00, 0.0000e+00]],

# #         [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
# #           0.0000e+00, 0.0000e+00],
# #          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
# #           0.0000e+00, 0.0000e+00]]], device='cuda:0',
# #        grad_fn=<SoftmaxBackward0>)
# # hidden_states tensor([[[ -1.1481, -13.1201, -15.8587,  ...,  -0.9798,  -4.3610, -10.1784],
# #          [ 13.2016, -24.5957,   7.8465,  ...,  -8.3187, -15.7784,  25.5586]],

# #         [[  0.5990, -13.2917,  -4.2307,  ...,  -1.4809,   6.1272, -13.4710],
# #          [ 20.9433, -20.0714,  -1.9486,  ...,  -9.3339, -23.8173,  12.3719]],

# #         [[  4.6366,  -8.0839,  -0.2394,  ...,  -2.3300,   8.7900, -21.3121],
# #          [ 37.4991, -15.1412,  -4.2812,  ...,  -3.4202, -24.2509,  10.7919]],

# #         ...,

# #         [[  3.9679,  -3.3238,  -6.0012,  ..., -23.8333,   1.5770, -15.0621],
# #          [-24.3199,  23.8817, -11.8673,  ...,   1.5562, -36.3175,  19.2838]],

# #         [[ 39.0543,  -9.2513, -16.8078,  ...,  16.1385, -23.5208,  -8.3467],
# #          [ -8.6503, -11.4497, -12.1152,  ..., -26.9105, -19.2700, -20.4602]],

# #         [[ 30.2774, -31.2572, -17.3581,  ...,  12.7839,  -4.1689, -18.0616],
# #          [ -2.3834,  -0.2117, -19.5934,  ..., -37.2427, -13.9567, -16.2300]]],
# #        device='cuda:0', grad_fn=<ViewBackward0>)
# # codevector_probs.shape [B * L, G, V]: torch.Size([1499, 2, 320])
# # codevector_probs sum over B*L [[  3   1   2   5   2   2   2   9   0   6   2   1   4   2   1   6   0   1
# #     2   1   6   0   1   1   3   2   2   4   5   2   2   2   5   1   2   3
# #     3   1   3   2   1   5   1   7   2   3   3   3   6   3   2   9   3   6
# #     2   2   3   3   3   7   4   2   4   2   1   3   4   8   2   2   1   5
# #     2   3   4   2   5   3   3   3   1   1   2   5   2   2   1   4   2   1
# #     5   6   1   1   2   0   3   6   1   4   2   4   6   1   3   1   4   1
# #     3   4   5   4   3   3   1   3   1   0   2   3   2   0   5   2   5   1
# #     1   2   3   4   1   3   4   2   2   3   3   1   3   1   4   5   2   3
# #     1   3   2  40   2   2   2   1   1   4   2   2   6   2 442   1   2   2
# #     1   1   2   3   2   2   1   1   0   2   1   5  12   1  16   2   5   2
# #     1   2   4   0   5   7   3   1   1   2   1   2   2   2   3   4   4   2
# #     2   6   6   2   1   3   2   4   2   1   1   2   3   4   4   3   3   4
# #     2   2   6   2   2   4   1   8   6   1  13 114   3   0   5   2   1   2
# #     2   1   3   3   2   3   2   2   5   0   1   2   5   1   3   2   8   2
# #     1   2   3   3   2   2   2  11   5   7   3   1   5   1   6   4   2   2
# #     0   2   1   3   1   1   2   2   1   4   1   6   5   2   1   3   2   1
# #     3   1   3  11   5   4   0   3   0   1   4   1   0   3   1   2   1   3
# #     3   3   4   2   2   2   4   2   3  12   0   4   8   3]
# #  [  1   5   2   3   2   1   3   3   2   1   2   3   3   2   3   0   2   3
# #     3   2   3   3   1   2   2   1   3   3   4   7   5   1   2   2   1   2
# #     1   0   2   3   2   4   3   1   2   3   2   3   3   4   2   2   4   2
# #     2   3   5   5   4   1   2   1   2   0   3   3   1   1   4   1   3   1
# # ...
# #  ...
# #  [1 1]
# #  [1 1]
# #  [1 1]]codevector_soft_dict tensor([[[6.7771e-04, 3.4817e-04, 2.9897e-04,  ..., 6.8408e-04,
#           5.6678e-04, 4.1008e-04],
#          [1.6854e-03, 2.0583e-04, 1.2512e-03,  ..., 5.0906e-04,
#           3.3616e-04, 3.3517e-03]],

#         [[1.8225e-04, 8.8679e-05, 1.4187e-04,  ..., 1.6361e-04,
#           2.4275e-04, 8.7859e-05],
#          [9.9370e-04, 1.1845e-04, 3.0318e-04,  ..., 2.0672e-04,
#           9.7541e-05, 6.3711e-04]],

#         [[1.0753e-04, 5.8837e-05, 8.5339e-05,  ..., 7.7287e-05,
#           1.3093e-04, 3.1428e-05],
#          [1.0111e-03, 8.3379e-05, 1.3952e-04,  ..., 1.4533e-04,
#           5.4139e-05, 2.8507e-04]],

#         ...,

#         [[2.0575e-03, 1.4064e-03, 1.2231e-03,  ..., 4.8241e-04,
#           1.8162e-03, 7.6235e-04],
#          [1.4706e-04, 1.8182e-03, 2.8161e-04,  ..., 5.6727e-04,
#           7.8642e-05, 1.4304e-03]],

#         [[6.0881e-03, 5.7449e-04, 3.9711e-04,  ..., 1.9867e-03,
#           2.8604e-04, 6.0045e-04],
#          [2.3699e-04, 2.0669e-04, 2.0007e-04,  ..., 9.7091e-05,
#           1.4104e-04, 1.3307e-04]],

#         [[1.1329e-03, 4.8084e-05, 9.8163e-05,  ..., 4.6142e-04,
#           1.9322e-04, 9.4680e-05],
#          [4.8687e-04, 5.4430e-04, 2.0121e-04,  ..., 8.1296e-05,
#           2.6874e-04, 2.3913e-04]]], device='cuda:0',
#        grad_fn=<SoftmaxBackward0>)
# codevector_probs.shape [B * L, G, V]: torch.Size([1499, 2, 320])
# codevector_probs sum over B*L [[ 8  5  6  5  5  6  4 11  2  4  2  8  5  5  3  2  4  4  4  4  4  5  7  5
#    4  6  7  3  3  4  5  3  4  4  7  1  4  3  7  3  3  4  4  6  6  6  2  4
#    4  4  5  8  3  6  2  8  1  3  3  3  6  6  6  8  6  3  7  6  4  5  5  2
#    9 10  5  1  4  3  8  3  3  5  1  4  5  8  3  2  5  4  4  5  4  5  4  2
#    2  4  4  7  4  7  7  0  5  2  5  4  3  2  4  3  5  5  4  5 11  2  2  5
#    4  4  1  4  3  2  3  5  3  7  3  3  5  4  4  8  6  4  4  1  7  4  6  1
#    2  3  6 12  4  2  6  3  2  5  4  2  3  6 11  4  7  7  3  4  3  6  7  2
#    8  6 10  5  7  6  8  3  6  6  7  6  8  2  6  7  2  0  6  5  4  8  6  3
#    2  2  1  6  6  4  7  6  5  6  5  3  8  4  5  4  9  3  6  3  3  6  7  6
#    5  6  4  6  4  5  5  5  3  4  3 11  2  1  5  5  4  2  3  6  5  7  2  5
#    7  5  7  6  3  4  7  4  4  6  4  6  6  5  7  5  6  3  4  3  4  6  3  3
#    6  3  4  7  5  3  6  5  5  8  5  5  6  6  5  3  3  8  2  4  3  5  5  6
#    3  2  3  3  6  3  5  6  5  6  2  4  5  6  6  7  5  8  7  2  4  5  6  5
#    2  6  6  1  3  4  5  8]
#  [ 6  5  6  5  6  3  4  3  5  4  3  5  4  7  3  7  5  4  5  5  1  4  7  3
#    5  3  6  7  4  4  3  4  4  2  6  6  5  4  3  4  5  7  7  7  7  1  6  5
#    7  5  5  3  4  3  6  5  5  6  6  1  4  5  4  4  7  1  2  5  5  4  5  6
#    7  1  5  4  2  2  4  6  2  5  6  4  7  5  3  5  4  3  9  6  4  3  6  0
#    3  5  5  1  4  5  3  4  4  4  4  3  3  7  6  7  1  7  5  5  3  8  7  5
#    5  5  4  1  5  5  6  3  4  2  2  3  5  6  5  4  7  4  8  6 10  5  4  2
#    5  6  7  4  3  4  3  6  3  4  9  1  3  6  4  4  6  9  4  6  4  4  5  2
#    4  5  2  7  4  7  4  5  6  6  5  4  6  3  1  4  7  2  2  5  4  4  3  3
#    3  5  7  6  4  2  4  6  3  3  4  5  1  4  7 15  3  5  4  7  5  6  6  6
#    5  4  6  8  5  2  6  7  4  5  1  5  6  4  4  3  4  4  4  6  4  3  1  3
#    6  1  9  5  8  4  3  4  6  3  6  5  2  4  9  4  4  8  8  6  6  6  6  6
#    3  9  8  1  3  5  6 10  5  6  4  8  6  6  2  6  2  4  5  5  2  6  6  3
#    5  6  2  7  2  4  8  6  5  2  6  5  3  3  7  3  6  3  8  7  6  3  7  1
#    4 13  4  3  2  5  4  3]]
# codevector_probs over V [[1 1]
#  [1 1]
#  [1 1]
#  ...
#  [1 1]
#  [1 1]
#  [1 1]]
# Wav2Vec2ForPreTrainingOutput(loss=None, projected_states=tensor([[[-1.7276, -0.3650, -0.4768,  ..., -0.9140, -0.4881,  0.0364],
#          [-1.1243, -0.2782, -0.6690,  ..., -1.0949, -0.2385, -0.2917],
#          [-1.4703, -0.0832, -1.0978,  ..., -1.0512, -0.7388, -0.0742],
#          ...,
#          [-1.6735, -0.4336, -0.5843,  ..., -1.3830, -0.3863,  0.1150],
#          [-1.3169,  0.1768,  0.1353,  ..., -1.3960,  0.0596,  0.1976],
#          [-0.6191,  0.0063,  0.4891,  ..., -1.3116, -0.0828, -0.0184]]],
#        device='cuda:0', grad_fn=<ViewBackward0>), projected_quantized_states=tensor([[[ 0.3143,  0.2197,  0.1293,  ..., -0.1033, -0.2783, -0.2106],
#          [ 0.1304,  0.6167,  0.6627,  ...,  0.1281, -0.2010,  0.0340],
#          [-0.0277,  0.5489,  0.1034,  ...,  0.0981,  0.0057,  0.0118],
#          ...,
#          [-0.1732,  0.0181, -0.2224,  ...,  0.5759, -0.3987,  0.0580],
#          [-0.3506, -0.4136, -0.0294,  ...,  0.6590, -0.4310,  0.4417],
#          [-0.0014, -0.1022, -0.1966,  ...,  0.0518, -0.2765,  0.0220]]],
#        device='cuda:0', grad_fn=<ViewBackward0>), codevector_perplexity=tensor(127.2690, device='cuda:0', grad_fn=<SumBackward1>), hidden_states=None, attentions=None, contrastive_loss=None, diversity_loss=None)

In [5]:
# model.model.quantizer.weight_proj.weight.mean(), model.model.quantizer.weight_proj.weight.std()

In [6]:
# sinkhorn
# model_name_or_path = '/scicore/home/dokman0000/liu0003/projects/seisLM/results/models/pretrained_seisLM/42__2024-06-17-07h-24m-11s_model_config_4xdownsample_sinkhorn/checkpoints/epoch=4-step=87350.ckpt'

# no sinkhorn
model_name_or_path = '/scicore/home/dokman0000/liu0003/projects/seisLM/results/models/pretrained_seisLM/42__2024-06-17-07h-24m-09s_model_config_4xdownsample/checkpoints/epoch=5-step=104820.ckpt'
model = pretrained_models.LitMultiDimWav2Vec2.load_from_checkpoint(
    model_name_or_path
)

model.train()
model.model(input_values = input_values.cuda())



codevector_probs.shape [B * L, G, V]: torch.Size([1499, 2, 320])
codevector_probs sum over B*L [[  0   0   0   0   0   0   0   0   0   0   0   0   0 404   0 100   0   0
    0   0   0   0   0   0   0   0   3   0   0   0   1   0   0   0   0   0
    0   0   0   1  39   0   0   0   0   0   0   0   0   0  11   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 126
    0   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   1   0
    6   0   2   0   0   0   0 201   0   0   0   0   0   0   0   0   0  16
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  18   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   1   0   0   0   0   0   0   0   0   0   0   3   0   0   0
    0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   0   0
    0   0   0   0

Wav2Vec2ForPreTrainingOutput(loss=None, projected_states=tensor([[[ 6.2135e-04, -5.8037e-04, -1.5818e-01,  ...,  4.0637e-01,
           7.4205e-01,  1.0916e+00],
         [ 5.3675e-01,  3.1447e-01, -9.1471e-02,  ...,  4.4546e-01,
           7.0710e-01,  1.3934e+00],
         [ 2.2042e-01, -5.9691e-01, -4.3454e-01,  ...,  1.1973e+00,
          -7.3712e-01,  1.8682e-01],
         ...,
         [ 1.0122e-01,  6.9160e-01, -2.2537e-01,  ...,  2.9373e-01,
          -1.2333e-01,  1.0156e+00],
         [ 1.4444e-01,  5.3106e-02, -6.8206e-01,  ...,  2.2695e-01,
          -5.2119e-02,  7.2586e-01],
         [ 1.2516e-01, -1.9080e-01, -9.1310e-01,  ...,  9.7079e-02,
          -9.9868e-01,  9.3790e-01]]], device='cuda:0',
       grad_fn=<ViewBackward0>), projected_quantized_states=tensor([[[ 0.2683,  0.4400,  0.1790,  ..., -0.4024, -0.1001,  0.0931],
         [ 0.0954, -0.0181, -0.1022,  ...,  0.4110, -0.1951,  0.0502],
         [-0.0243,  0.0757, -0.1856,  ...,  0.3650, -0.2689, -0.0950],
       